## Batch Normalization

 Batch Normalization is really an optimization to help train faster not specifically used to make the network better.
 
Making normalization as a part of the model architecture and performing the normalization for each training mini-batch. For example the current batch is at 32 so it performed normalization per 32.

### [Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift](https://arxiv.org/pdf/1502.03167.pdf)

"Training Deep Neural Networks is complicated by the fact that the distribution of each layer’s inputs changes during training, as the parameters of the previous layers change. This slows down the training by requiring lower learning rates and careful parameter initialization, and makes it notoriously hard to train models with saturating nonlinearities. We refer to this phenomenon as internal covariate shift, and address the problem by normalizing layer inputs."

As the data flows through a deep network, the weights and parameters adjust those values, sometimes making the data too big or too small again - a problem the authors refer to as "__internal covariate shift__". By normalizing the data in each mini-batch, this problem is largely avoided. Batch Normalization normalizes each batch by both mean and variance reference.

Batch normalization ensures the distribution of nonlinearity inputs remains more stable as the network trains since the optimizer would be less likely to get stuck in the saturated level, for example, sigmoid activation (~3), and the training would accelerate.

The same saturation problem and the resulting vanishing gradients can also be addressed by using Rectified Linear Unit (ReLu) Max(0, x).

### Benefits

- Networks train faster converge much more quickly
- Allows higher learning rates since gradient descent usually requires small learning rates for the network to converge
- Makes weights easier to initialize
- Makes more activation functions viable because batch normalization regulates the values going into each activation function, non-linearities that don't seem to work well in deep networks actually become viable again
- May give better results overall
- Batch Normalization allows us to use much higher learning rates and be less careful about initialization
- It also acts as a regularizer, in some cases eliminating the need for Dropout


### DEBATE: Normalization before Activation vs. Normalization after Activation

TODO: Add notes

- Current standard is that you add normalization before calling the activation function

    - Regular dense layer
        - model.add(layers.Dense(64, activation='relu'))
    
    - Tell dense layer not to use the bias since it's not needed
        - model.add(layers.Dense(64, use_bias=False))
        - model.add(layers.BatchNormalization())
        - model.add(Activation("relu"))
        
    
- Conv2D Layer Example

    - Use a Conv2D layer without the bias node
        - model.add(layers.Conv2D(64, (3, 3), use_bias=False))
        - model.add(layers.BatchNormalization())
        - model.add(layers.Activation("relu"))


#### Set scale=False, because the next layer is linear (ReLU)

x = Conv2D(n_filters, kernel, padding=padding, use_bias=False)(inputs)

x = BatchNormalization(axis=-1, center=True, scale=False)(x)

x = ReLU()(x)

BatchNormalization layer followed by linear or relu you can skip the the scale operation.

#### Another example of Conv -> ReLU -> Pooling

#### CONV => RELU => POOL
model.add(Conv2D(32, (3, 3), padding="same", input_shape=inputShape)) # 32 filters with 3x3 kernel

model.add(Activation("relu"))

model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(3, 3))) # reduce special dimensions

model.add(Dropout(0.25))

### Reducing Covariate Shift - Paper text

We define Internal Covariate Shift as the change in the distribution of network activations due to the change in network parameters during trainin 

To improve the training, we seek to reduce the internal covariate shift. By fixing the distribution of the layer inputs x as the training progresses, we expect to improve the training speed. It has been long known (LeCun et al., 1998b;  Wiesler & Ney, 2011) that the network training __converges faster__ if its inputs are whitened – i.e., linearly transformed to have zero means and unit variances, and decorrelated. As each layer observes the inputs produced by the layers below, it would be advantageous to achieve the same whitening of the inputs of each layer. 

By whitening the inputs to each layer,we would take a step towards achieving the fixed distributions of inputs that would remove the ill effects of the internal covariate shift

## Mini-batches

We use mini-batches in stochastic gradient training, each mini-batch produces estimates of the mean and variance of each activation. This way, the statistics used for normalization can fully  participate in the gradient backpropagation.

### BN vs. Standardization

Excerpt from the paper.

The goal of Batch Normalization is to achieve a stable distribution of activation values throughout training, and in our experiments __we apply it before the nonlinearity__ since that is where matching the first and second moments is more  likely to result in as table distribution. On the contrary, (G  ̈ulc ̧ehre & Bengio, 2013) apply the standardization layer to the output of the nonlinearity, which results in sparser activations.  

In our large-scale image classification experiments, we have not observed the nonlinearity inputs to be sparse, neither within or without Batch Normalization. Other notable differentiating characteristics of Batch Normalization include the learned scale and shift that allow the BN transform to represent identity (the standardization layer did not require this since it was followed by the learned linear transform that, conceptually, absorbs the necessary scale and shift), handling of convolutional layers, deterministic inference that does not depend on the mini-batch, and batch-normalizing each convolutional layer in the network.

--------

### Article Notes - [Link](https://towardsdatascience.com/batch-normalization-theory-and-how-to-use-it-with-tensorflow-1892ca0173ad)

In order to understand what batch normalization is, first we need to address which problem it is trying to solve.

Usually, in order to train a neural network, we do some preprocessing to the input data. For example, we could normalize all data so that it resembles a normal distribution (that means, zero mean and a unitary variance). Why do we do this preprocessing? Well, there are many reasons for that, some of them being: preventing the early saturation of non-linear activation functions like the sigmoid function, assuring that all input data is in the same range of values, etc.

But the problem appears in the intermediate layers because the distribution of the activations is constantly changing during training. This slows down the training process because each layer must learn to adapt themselves to a new distribution in every training step. This problem is known as internal covariate shift.